In [1]:
from agnews_utils import get_data, tokenizing, get_matrix
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
import numpy as np

# matrix_path = 's3://obs-wzhtest/cifar/glove.6B/glove.6B.100d.txt'
matrix_path = 's3://obs-wzhtest/cifar/glove.6B/glove.6B.100d.txt'
train_data_path = 'ag-news/ag_news_csv/train.csv'
test_data_path = 'ag-news/ag_news_csv/test.csv'

max_num_words = 20000
max_sequence_length = 150
embedding_dim = 100
batch_size = 500
epochs = 3

x_train, y_train, x_test, y_test = get_data(train_data_path, test_data_path)
word_index, x_train, y_train, x_test, y_test = tokenizing(x_train, y_train, x_test, y_test, max_num_words, max_sequence_length)
# embedding_matrix = get_matrix(matrix_path, word_index, max_num_words, embedding_dim)

INFO:root:Using MoXing-v1.14.1-d1850ed3
INFO:root:Using OBS-Python-SDK-3.1.2
Using TensorFlow backend.


Loading data complete.
shape of x_train tensor:  (120000, 150)
shape of y_train tensor:  (120000, 4)
shape of x_test tensor:  (7600, 150)
shape of y_test tensor:  (7600, 4)
Tokenizing complete.


In [4]:
# 1. 这里用到TextRNN手段对agnews进行训练

from model.agnews_model import TextRNN

print('Build model...')
adam = optimizers.Adam(lr=0.05)
model = TextRNN(max_sequence_length, max_num_words, embedding_dim)
model.compile(loss='categorical_crossentropy', optimizer=adam,
             metrics=['accuracy'])
print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))
print('Test...')
result = model.predict(x_test)
result_label = np.argmax(result, 1)
y_label = np.argmax(y_test, 1)
print('result label: ', result_label)
print('y_label: ', y_label)
print('test set accuracy: ', np.equals(result_label, y_label).mean())

Build model...
Train...
Train on 120000 samples, validate on 7600 samples
Epoch 1/3
120000/120000 [==============================] - 59s 495us/step - loss: 0.4720 - acc: 0.8383 - val_loss: 0.3905 - val_acc: 0.8766
Epoch 2/3


/home/ma-user/anaconda3/envs/TensorFlow-1.8/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


120000/120000 [==============================] - 53s 438us/step - loss: 0.3515 - acc: 0.8859 - val_loss: 0.3942 - val_acc: 0.8770
Epoch 3/3
120000/120000 [==============================] - 53s 438us/step - loss: 0.3459 - acc: 0.8870 - val_loss: 0.4068 - val_acc: 0.8657
Test...
result label:  [1 0 3 ... 2 3 0]
y_label:  [3 0 0 ... 2 3 3]


In [7]:
# 2. 这里使用TextBiRNN对agnews进行训练

from model.agnews_model import TextBiRNN

print('Build model...')
adam = optimizers.Adam(lr=0.05)
model = TextBiRNN(max_sequence_length, max_num_words, embedding_dim)
model.compile(adam, 'categorical_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result_label = np.argmax(result, 1)
y_label = np.argmax(y_test, 1)
print('test set accuracy: ', np.equal(result_label, y_label).mean())

Build model...
Train...
Train on 120000 samples, validate on 7600 samples
Epoch 1/3
120000/120000 [==============================] - 104s 865us/step - loss: 0.7128 - acc: 0.7295 - val_loss: 0.4263 - val_acc: 0.8554
Epoch 2/3
120000/120000 [==============================] - 100s 834us/step - loss: 0.3758 - acc: 0.8775 - val_loss: 0.4108 - val_acc: 0.8655
Epoch 3/3
120000/120000 [==============================] - 100s 837us/step - loss: 0.3602 - acc: 0.8822 - val_loss: 0.4147 - val_acc: 0.8629
Test...
test set accuracy:  0.8628947368421053


In [15]:
# 3. 这里使用TextCNN进行训练

from model.agnews_model import TextCNN

epochs = 5

print('Build model...')
model = TextCNN(max_sequence_length, max_num_words, embedding_dim)
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])
# model.compile(adam, 'binary_crossentropy', metrics=['accuracy'])

print('Train...')
early_stopping = EarlyStopping(monitor='val_acc', patience=3, mode='max')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping],
          validation_data=(x_test, y_test))

print('Test...')
result = model.predict(x_test)
result_label = np.argmax(result, 1)
y_label = np.argmax(y_test, 1)
print('result label: ', result_label[35:55])
print('y label:      ', y_label[35:55])
print('-----------------------------------------------')
print('result label: ', result_label[135:155])
print('y label:      ', y_label[135:155])
print('test set accuracy: ', np.equal(result_label, y_label).mean())

Build model...
Train...
Train on 120000 samples, validate on 7600 samples
Epoch 1/5
120000/120000 [==============================] - 10s 80us/step - loss: 0.2045 - acc: 0.9195 - val_loss: 0.1230 - val_acc: 0.9581
Epoch 2/5
120000/120000 [==============================] - 9s 73us/step - loss: 0.0851 - acc: 0.9711 - val_loss: 0.1113 - val_acc: 0.9609
Epoch 3/5
120000/120000 [==============================] - 9s 73us/step - loss: 0.0490 - acc: 0.9847 - val_loss: 0.1119 - val_acc: 0.9621
Epoch 4/5
120000/120000 [==============================] - 9s 74us/step - loss: 0.0266 - acc: 0.9929 - val_loss: 0.1151 - val_acc: 0.9625
Epoch 5/5
120000/120000 [==============================] - 9s 73us/step - loss: 0.0144 - acc: 0.9967 - val_loss: 0.1221 - val_acc: 0.9627
Test...
result label:  [1 3 1 1 1 2 3 1 0 2 2 1 0 1 2 1 2 1 0 3]
y label:       [1 1 1 1 1 2 3 1 0 2 2 1 0 1 2 1 2 1 0 3]
-----------------------------------------------
result label:  [2 1 2 2 1 2 1 2 1 1 1 2 3 2 2 2 1 2 1 1]
y label:

In [16]:
print('result label: ', result_label[235:255])
print('y label:      ', y_label[235:255])

result label:  [2 3 2 1 3 3 2 2 0 3 3 0 0 3 1 3 1 2 3 3]
y label:       [2 3 2 1 3 0 2 2 0 3 3 0 0 0 1 3 1 2 3 3]
